In [1]:
!pip install llama-index-llms-huggingface

In [2]:
!pip install llama-index

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [4]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()

In [6]:
print(documents)

[Document(id_='4d9975f8-624e-4992-8c1c-6752f48873dd', embedding=None, metadata={'page_label': '1', 'file_name': 'ULM.pdf', 'file_path': '/content/data/ULM.pdf', 'file_type': 'application/pdf', 'file_size': 421696, 'creation_date': '2024-07-18', 'last_modified_date': '2024-07-18'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Uniﬁed Language Model Pre-training for\nNatural Language Understanding and Generation\nLi Dong∗Nan Yang∗Wenhui Wang∗Furu Wei∗†Xiaodong Liu Yu Wang\nJianfeng Gao Ming Zhou Hsiao-Wuen Hon\nMicrosoft Research\n{lidong1,nanya,wenwan,fuwei}@microsoft.com\n{xiaodl,yuwan,jfgao,mingzhou,hon}@microsoft.com\nAbstract\nThis paper presents a new UNIﬁed pre-trained Language Model ( UNILM) that\ncan be ﬁne-tuned for both natural language unders

In [7]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [8]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1.
401 Client Error. (Request ID: Root=1-6698d31f-48a3639e1a8875860b8d77bb;52d75d96-9bc7-48f0-a6d7-3327fe477278)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must be authenticated to access it.

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model =HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [ ]:
from llama_index.core import VectorStoreIndex, ServiceContext

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
query_engine.query("what is attention?")

In [ ]:
query_engine.query("how attention is different from rnn and lstm")